<a href="https://colab.research.google.com/github/rittikarijhwani/ML-lab-sem7/blob/main/exp_7_ensemble_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#ready
import pandas as pd

df = pd.read_csv('pakistan_hunger_data.csv')
df.head()

,City,Year,Population_Under_Poverty,Malnutrition_Rate,Food_Insecurity,Access_to_Clean_Water,Food_Production_Index,Children_Underweight
0,Quetta,2021,23.8,23.0,22.0,82.2,95.1,31.7
1,Hyderabad,2023,29.6,17.7,23.3,82.7,90.8,27.5
2,Rawalpindi,2020,25.3,23.0,24.4,89.0,99.6,25.8
3,Karachi,2021,25.4,19.3,20.6,87.5,98.3,25.4
4,Peshawar,2022,27.5,16.2,21.5,84.8,96.2,33.7


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

df = pd.read_csv('pakistan_hunger_data.csv')

#define features and target variable
X = df[['Population_Under_Poverty', 'Malnutrition_Rate', 'Food_Insecurity', 'Access_to_Clean_Water']]
y = df['Children_Underweight']

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def calculate_metrics(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, r2

#models for stacking, bagging, and boosting

#stacking regressor (2 base models: Logistic Regression, SVM, with LR meta model)
base_models = [
    ('lr', LinearRegression()),
    ('svr', SVR())
]

meta_model = LinearRegression()

stacking_reg = StackingRegressor(estimators=base_models, final_estimator=meta_model)

stacking_reg.fit(X_train, y_train) #training stacking model

#testing and metrics for stacking
y_pred_stacking = stacking_reg.predict(X_test)
mse_stack, mae_stack, r2_stack = calculate_metrics(y_test, y_pred_stacking)


#bagging - random forest with 10 decision trees
bagging_reg = RandomForestRegressor(n_estimators=10, random_state=42)
bagging_reg.fit(X_train, y_train)

#testing and metrics for bagging
y_pred_bagging = bagging_reg.predict(X_test)
mse_bagging, mae_bagging, r2_bagging = calculate_metrics(y_test, y_pred_bagging)


#boosting - XGBoost with 10 trees
boosting_reg = XGBRegressor(n_estimators=10, random_state=42)
boosting_reg.fit(X_train, y_train)

#testing and metrics for boosting
y_pred_boosting = boosting_reg.predict(X_test)
mse_boosting, mae_boosting, r2_boosting = calculate_metrics(y_test, y_pred_boosting)

#comparison table
comparison = pd.DataFrame({
    'Model': ['Stacking', 'Bagging (Random Forest)', 'Boosting (XGBoost)'],
    'MSE': [mse_stack, mse_bagging, mse_boosting],
    'MAE': [mae_stack, mae_bagging, mae_boosting],
    'R²': [r2_stack, r2_bagging, r2_boosting]
})

print(comparison)

                     Model       MSE       MAE        R²
0                 Stacking  8.462002  2.496124 -0.011128
1  Bagging (Random Forest)  8.496149  2.479900 -0.015208
2       Boosting (XGBoost)  9.010076  2.518373 -0.076617
